# Basic wowp usage

## Turning functions into actors

### 1. Define a function

We will define a simple `times2` function. Annotations (see [PEP 3107](https://www.python.org/dev/peps/pep-3107/)) will be used for output port names.

In [1]:
def times2(x) -> ('y'):
    '''Multiplies the input by 2
    '''
    return x * 2

### 2. Turn the function into an *actor*

In [2]:
from wowp.actors import FuncActor

times2_actor = FuncActor(times2)

### 3. Inspect actor's input and output ports

Input / output ports are accessible via `inports` and `outports` properties.

In [3]:
print('input  ports: {}'.format(times2_actor.inports.keys()))
print('output ports: {}'.format(times2_actor.outports.keys()))

input  ports: ['x']
output ports: ['y']


### 4. FuncActor is callable

In [4]:
x = 3
print('times2({}) = {}'.format(x, times2(x)))
print('times2_actor({}) = {}'.format(x, times2_actor(x)))
assert times2(x) == times2_actor(x)

times2(3) = 6
times2_actor(3) = 6


## Simple workflows

1. Workflows are created by connecting actor ports (input ports to output ports).
2. Ports get connected using the **`+=`** operator (`inport += outport`).

*Better workflow creation will be implemented soon. `Actor.get_workflow` will create a workflow *automagically*. It will also be possible to create wokflows *explicitely*, e.g. in cases when `get_workflow` cannot be used.* 

### Two actors chained together

Let's try something like `x -> actor1 -> actor2 -> out`.

In [5]:
# create two FuncActors
actor1 = FuncActor(lambda x: x * 2)
actor2 = FuncActor(lambda x: x + 1)

# chain the actors
# FuncActor output port is by default called out
actor2.inports['x'] += actor1.outports['out']

Get the resulting workflow.

In [6]:
wf = actor1.get_workflow()

Execute the workflow just like an actor.

In [7]:
wf(x=3)

{'out': deque([7])}

## Creating a custom actor

In [8]:
from wowp import Actor

Every actor must implement `get_run_args` and `run` methods:
* `get_run_args` returns an (args, kwargs) tuple for the later `run(*args, **kwargs)` call. This method is responsible for getting (popping) values from input ports. `args` and `kwargs` needs to be serializable for subprocess-based schedulers (e.g. IPython cluster).
* The `run` method gets the input arguments returned by `get_run_args`. The output must be a dictionary with output port names as keys. `run` must be decorated by `@staticmethod` or `@classmethod` in order to be serializable---this is necessary for subprocess-based schedulers (e.g. IPython cluster).
* The result of `run` must be a `dict` (like) object, whose keys are output port names.
Optional, these methods might be overridden:
* `can_run` returns True if the actor is ready to be run (usually when it has received enough inputs). `can_run` is called whenever a new input arrives (on an input port). By default, `can_run` waits for values on all connected ports.


In [9]:
class StrActor(Actor):

    def __init__(self, *args, **kwargs):
        super(StrActor, self).__init__(*args, **kwargs)
        # specify input port
        self.inports.append('input')
        # and output ports
        self.outports.append('output')
        
    def get_run_args(self):
        # get input value(s) using .pop()
        args = (self.inports['input'].pop(), )
        kwargs = {}
        return args, kwargs

    @staticmethod
    def run(value):
        # return a dictionary with port names as keys
        res = {'output': str(value)}
        return res

Create an instance.

In [10]:
actor = StrActor(name='str_actor')

Test the actor by direct call.

In [11]:
# we can call the actor directly -- see what's output
value = 123
print(actor(input=value))
# and check that the output is as expected
assert actor(input=value)['output'] == str(value)

{'output': '123'}
